In [ ]:
%run C:/Users/cosmosyw/Documents/Softwares/DNA_MERFISH_analysis/Startup.py

import ImageAnalysis3 as ia

from ImageAnalysis3 import *
print(os.getpid())

from sklearn.decomposition import PCA

In [ ]:
codebook = pd.read_csv(r'\\10.245.74.158\Chromatin_NAS_8\Exported_data\20220713-Export\merged_codebook.csv')

postanalysis_folder = r'E:/DNA_analysis/Postanalysis_MeCP2/'
if not os.path.exists(postanalysis_folder):
    os.mkdir(postanalysis_folder)

subclass_2_median_filename = os.path.join(postanalysis_folder, 'subclass_2_medianDict.pkl')
print(subclass_2_median_filename)

if os.path.exists(subclass_2_median_filename):
    print("Loading")
    subclass_2_medianDict = np.load(subclass_2_median_filename, allow_pickle=True)
    
import ImageAnalysis3.structure_tools.distance as distance
codebook_df = codebook.copy()

In [ ]:
sel_subclasses = ['Oligo', 'L5 IT', 'Micro', 'Peri', 'Endo-PVM', 'Astro', 'OPC', 'L6 CT', 'L5 ET', 'L5/6 NP', 'Pvalb', 'L6 IT', 'Lamp5', 'L6b', 'Sst', 'L4/5 IT', 'L2/3 IT', 'Vip']

In [ ]:
figure_parent_folder = r'E:\DNA_analysis\Postanalysis_MeCP2\AB_by_median\subclass_MOp'
if not os.path.exists(figure_parent_folder):
    os.mkdir(figure_parent_folder)

In [ ]:
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
from scipy.spatial.distance import euclidean

_dpi = 300 # dpi required by figure
_single_col_width = 2.25 # figure width in inch if occupy 1 colomn
_double_col_width = 4.75 # figure width in inch if occupy 1 colomn
_single_row_height= 2 # comparable height to match single-colomn-width
_ticklabel_size=2
_ticklabel_width=0.5
_font_size=7.5

In [ ]:
# file IO
figure_folder = os.path.join(figure_parent_folder, 'figure')
if not os.path.exists(figure_folder):
    os.mkdir(figure_folder)

array_folder = os.path.join(figure_parent_folder, 'npy')
if not os.path.exists(array_folder):
    os.mkdir(array_folder)

In [ ]:
cell_type_conversion = {'L4/5 IT':'L45 IT', 'L2/3 IT':'L23 IT', 'L5/6 NP':'L56 NP'}

# load CG density

# load CG density
df_CG = pd.read_csv(r"E:\DNA_analysis\refgen_with_loci_index_CG_Density.csv")
df_CG.sort_values('loci_index', inplace=True)
df_CG.reset_index(inplace=True, drop=True)
df_CG

In [ ]:
def normalize_similarity_matrix(similarity_mat_input):
    min_val = np.percentile(similarity_mat_input[np.where(similarity_mat_input!=0)],1)
    max_val = np.percentile(similarity_mat_input[np.where(similarity_mat_input!=0)],99)
    similarity_mat = (similarity_mat_input-min_val)/(max_val-min_val)
    similarity_mat = np.clip(similarity_mat, 0 , 1)
    return similarity_mat

In [ ]:
from scipy.stats import pearsonr

dict_subclass_pc_values = {}
dict_subclass_pc_values['loci_name'] = df_subclass_ab['loci_name'].values

for sel_type in sel_subclasses:
    
    fig = plt.figure(figsize=(4,6), dpi=300)
    
    if sel_type in cell_type_conversion.keys():
        output_type = cell_type_conversion[sel_type]
    else:
        output_type = sel_type
    
    wt_npy_file = os.path.join(array_folder, f'{output_type}_WT_similarity_matrix.npy')
    ko_npy_file = os.path.join(array_folder, f'{output_type}_KO_similarity_matrix.npy')
    if os.path.exists(wt_npy_file):
        wt_similarity_mat = np.load(wt_npy_file)
    else:
        summary_dict = subclass_2_medianDict[sel_type+'_WT']
        sel_codebook = codebook_df
        sort_by_region = False
        # calculate similarity matrix
        _trans_mat_wt, chr_edges, chr_names = distance.assemble_ChrDistDict_2_Matrix(
            summary_dict, codebook_df, 
            sel_codebook=sel_codebook, 
            use_cis=False, use_trans=True, sort_by_region=sort_by_region,
        )
        _trans_mat_wt = _trans_mat_wt[:1981, :1981].copy()
        norm_trans_wt = _trans_mat_wt.copy()
        for i in range(chr_edges.shape[0]-2):
            col_start = chr_edges[i]
            col_end = chr_edges[i+1]

            for j in range(chr_edges.shape[0]-2):
                row_start = chr_edges[j]
                row_end = chr_edges[j+1]

                temp_mat = _trans_mat_wt[row_start:row_end, col_start:col_end]
                temp_norm = np.nanmean(temp_mat, axis=1)

                norm_trans_wt[row_start:row_end, col_start:col_end] = temp_mat/temp_norm.reshape((temp_norm.shape[0],1))
        # generate similarity matrix
        wt_similarity_mat = norm_trans_wt.copy()

        for i in range(wt_similarity_mat.shape[0]):
            for j in range(wt_similarity_mat.shape[1]):
                x = norm_trans_wt[:,i].copy()
                y = norm_trans_wt[:,j].copy()
                mask = (~np.isnan(x))&(~np.isinf(x))&(~np.isnan(y))&(~np.isinf(y))
                x = x[mask]
                y = y[mask]
                wt_similarity_mat[i,j] = euclidean(x, y)
        # save
        np.save(wt_npy_file, wt_similarity_mat)
    ### KO files
    if os.path.exists(ko_npy_file):
        ko_similarity_mat = np.load(ko_npy_file)
    else:
        summary_dict = subclass_2_medianDict[sel_type+'_KO']
        sel_codebook = codebook_df
        sort_by_region = False
        # calculate similarity matrix
        _trans_mat_wt, chr_edges, chr_names = distance.assemble_ChrDistDict_2_Matrix(
            summary_dict, codebook_df, 
            sel_codebook=sel_codebook, 
            use_cis=False, use_trans=True, sort_by_region=sort_by_region,
        )
        _trans_mat_wt = _trans_mat_wt[:1981, :1981].copy()
        norm_trans_wt = _trans_mat_wt.copy()
        for i in range(chr_edges.shape[0]-2):
            col_start = chr_edges[i]
            col_end = chr_edges[i+1]

            for j in range(chr_edges.shape[0]-2):
                row_start = chr_edges[j]
                row_end = chr_edges[j+1]

                temp_mat = _trans_mat_wt[row_start:row_end, col_start:col_end]
                temp_norm = np.nanmean(temp_mat, axis=1)

                norm_trans_wt[row_start:row_end, col_start:col_end] = temp_mat/temp_norm.reshape((temp_norm.shape[0],1))
        # generate similarity matrix
        ko_similarity_mat = norm_trans_wt.copy()

        for i in range(ko_similarity_mat.shape[0]):
            for j in range(ko_similarity_mat.shape[1]):
                x = norm_trans_wt[:,i].copy()
                y = norm_trans_wt[:,j].copy()
                mask = (~np.isnan(x))&(~np.isinf(x))&(~np.isnan(y))&(~np.isinf(y))
                x = x[mask]
                y = y[mask]
                ko_similarity_mat[i,j] = euclidean(x, y)
        # save
        np.save(ko_npy_file, ko_similarity_mat)
    
    # get rid off X chromosomes and define all parameters
    wt_similarity_mat = wt_similarity_mat[:1917, :1917].copy()
    ko_similarity_mat = ko_similarity_mat[:1917, :1917].copy()
    
    # concatenate and normalize
    wt_similarity_mat = normalize_similarity_matrix(wt_similarity_mat)
    ko_similarity_mat = normalize_similarity_matrix(ko_similarity_mat)
    similarity_mat = np.concatenate((wt_similarity_mat, ko_similarity_mat), axis=1)
    
    
    # PCA on similarity matrices - WT
    pca = PCA(n_components=3)
    pca.fit(wt_similarity_mat)
    pc1 = pca.components_[0]
    norm_pc1_wt = pc1/np.nanstd(pc1)
    pc2 = pca.components_[1]
    norm_pc2_wt = pc2/np.nanstd(pc2)
    
    # figure out the sign
    CG_density = df_CG.CG_density_1Mb.values
    r, p = pearsonr(norm_pc1_wt, CG_density)
    if r<0:
        norm_pc1_wt = -norm_pc1_wt
        
    # PCA on similarity matrices - KO
    pca = PCA(n_components=3)
    pca.fit(ko_similarity_mat)
    pc1 = pca.components_[0]
    norm_pc1_ko = pc1/np.nanstd(pc1)
    pc2 = pca.components_[1]
    norm_pc2_ko = pc2/np.nanstd(pc2)
    
    # figure out the sign
    CG_density = df_CG.CG_density_1Mb.values
    r, p = pearsonr(norm_pc1_ko, CG_density)
    if r<0:
        norm_pc1_ko = -norm_pc1_ko
        
    norm_pc1 = np.concatenate((norm_pc1_wt, norm_pc1_ko), axis=0)
    norm_pc2 = np.concatenate((norm_pc2_wt, norm_pc2_ko), axis=0)
    
    dict_subclass_pc_values[sel_type+'_WT'] = norm_pc1[:1917]
    dict_subclass_pc_values[sel_type+'_KO'] = norm_pc1[1917:]
    
    # plot
    ax1 = fig.add_subplot(111)
    # create a color map
    current_cmap = cm.get_cmap('seismic_r').copy()
    current_cmap.set_bad(color=[0.5,0.5,0.5,1])
    
    # common parameters
    chr_edges = np.array([   0,  153,  301,  424,  546,  665,  779,  875,  977, 1079, 1196,
           1302, 1380, 1472, 1566, 1644, 1720, 1790, 1858, 1917])
    chr_names = []
    for i in range(1,20):
        chr_names.append('chr'+str(i))
    #plot
    _pf = ax1.imshow(similarity_mat, 
                     cmap=current_cmap, vmin=0, vmax=0.4)
    ax1.set_title(sel_type, fontsize=7)
    ax1.set_yticks((chr_edges[1:] + chr_edges[:-1])/2, chr_names, fontsize=6)
    ax1.plot([1917, 1917], [0, 1916], color='black', linewidth=0.75)
    # locate ax1
    divider = make_axes_locatable(ax1)
   
    # colorbar ax
    cax = divider.append_axes('right', size='2%', pad="2%")
    cbar = plt.colorbar(_pf,cax=cax, ax=ax1, ticks=[0,0.2,0.4])
    cbar.ax.set_yticklabels([0,0.2,0.4])
    cbar.ax.tick_params('both', labelsize=_font_size-1, 
                    width=_ticklabel_width, length=_ticklabel_size-0.5,
                    pad=1, labelleft=False) # remove bottom ticklabels for ax1
    [i[1].set_linewidth(_ticklabel_width) for i in cbar.ax.spines.items()]
    #cbar.set_ticks([vmin,vmax])
    cbar.outline.set_linewidth(_ticklabel_width)
    cbar.set_label('Similarity for trans distance', 
                   fontsize=_font_size, labelpad=2, rotation=270)
    cbar.ax.minorticks_off()
    # create bottom ax
    bot_ax = divider.append_axes('bottom', size='10%', pad="2%", 
                                 sharex=ax1, xticks=[])
    bot_ax.bar(np.where(norm_pc1>0)[0], norm_pc1[norm_pc1>0], color='r', width=1, bottom=0)
    bot_ax.bar(np.where(norm_pc1<=0)[0], norm_pc1[norm_pc1<=0],color='b', width=1, bottom=0)

    bot_ax.tick_params('x', labelsize=_font_size-1, 
                    width=_ticklabel_width, length=_ticklabel_size,
                    pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
    bot_ax.tick_params('y', labelsize=_font_size-1, 
                    width=_ticklabel_width, length=0,
                    pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
    bot_ax.set_ylabel('PC1', fontsize=_font_size-1, labelpad=1, loc='top', rotation='horizontal')
    bot_ax.yaxis.set_label_coords(-0.01, .2)
    bot_ax.plot([1917, 1917], [-2, 2], color='black', linewidth=0.75)
    
    # create bottom ax
    bot_ax = divider.append_axes('bottom', size='10%', pad="2%", 
                                 sharex=ax1, xticks=[])
    bot_ax.bar(np.where(norm_pc2>0)[0], norm_pc2[norm_pc2>0], color='r', width=1, bottom=0)
    bot_ax.bar(np.where(norm_pc2<=0)[0], norm_pc2[norm_pc2<=0],color='b', width=1, bottom=0)

    bot_ax.tick_params('x', labelsize=_font_size-1, 
                    width=_ticklabel_width, length=_ticklabel_size,
                    pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
    bot_ax.tick_params('y', labelsize=_font_size-1, 
                    width=_ticklabel_width, length=0,
                    pad=1, labelleft=False, labelbottom=True) # remove bottom ticklabels for ax1
    bot_ax.set_ylabel('PC2', fontsize=_font_size-1, labelpad=1, loc='top', rotation='horizontal')
    bot_ax.yaxis.set_label_coords(-0.01, .2)
    bot_ax.plot([1917, 1917], [-2, 10], color='black', linewidth=0.75)
    
    plt.savefig(os.path.join(figure_folder,  f'{output_type}_AB.pdf'))
    plt.savefig(os.path.join(figure_folder,  f'{output_type}_AB.png'))
    

In [ ]:
df_subclass_Mecp2_PC = pd.DataFrame(dict_subclass_pc_values)
df_subclass_Mecp2_PC

In [ ]:
df_subclass_Mecp2_PC.to_csv(r'result\Mecp2_subclass_PC_values_call_Separate.csv')